<a href="https://colab.research.google.com/github/phongpirik/CS2225.CH1501/blob/tungnqd/remover_background/remover_Background.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install absl-py==0.9.0
!pip install astunparse==1.6.3
!pip install cachetools==4.1.1
!pip install certifi==2020.6.20
!pip install chardet==3.0.4
!pip install cycler==0.10.0
!pip install decorator==4.4.2
!pip install django==3.0.8
!pip install filelock==3.0.12
!pip install future==0.18.2
!pip install gast==0.3.3
!pip install gdown==3.11.1
!pip install gluoncv==0.7.0
!pip install google-auth==1.18.0
!pip install google-auth-oauthlib==0.4.1
!pip install google-pasta==0.2.0
!pip install graphviz==0.8.4
!pip install grpcio==1.30.0
!pip install h5py==2.10.0
!pip install idna==2.10
!pip install imageio==2.9.0
!pip install Keras-Preprocessing==1.1.2
!pip install kiwisolver==1.2.0 
!pip install Markdown==3.2.2
!pip install matplotlib==3.2.2
!pip install mxnet==1.6.0
!pip install networkx==2.4
!pip install numpy==1.19.0
!pip install oauthlib==3.1.0
!pip install opencv-python==4.3.0.36
!pip install opt-einsum==3.2.1
!pip install Pillow==7.2.0
!pip install portalocker==1.7.0 
!pip install protobuf==3.12.2
!pip install pyasn1==0.4.8
!pip install pyasn1-modules==0.2.8
!pip install pyparsing==2.4.7
!pip install PySocks==1.7.1
!pip install python-dateutil==2.8.1
!pip install PyWavelets==1.1.1
!pip install requests==2.24.0
!pip install requests-oauthlib==1.3.0
!pip install rsa==4.6
!pip install scikit-image==0.17.2
!pip install scipy==1.4.1
!pip install six==1.15.0
!pip install tensorboard==2.2.2
!pip install tensorboard-plugin-wit==1.7.0
!pip install tensorflow==2.2.1
!pip install tensorflow-estimator==2.2.0
!pip install termcolor==1.1.0
!pip install tifffile==2020.7.4
!pip install torch==1.5.1
!pip install tqdm==4.47.0
!pip install urllib3==1.25.9
!pip install Werkzeug==1.0.1
!pip install wrapt==1.12.1

     |████████████████████████████████| 112kB 10.7MB/s 
  Created wheel for absl-py: filename=absl_py-0.9.0-cp36-none-any.whl size=121933 sha256=1eda9fe805ed35263ecb80c9cb2cf1db681a7f8e16631f599f3227b566d674ea
  Stored in directory: /root/.cache/pip/wheels/8e/28/49/fad4e7f0b9a1227708cbbee4487ac8558a7334849cb81c813d
Successfully built absl-py
  Found existing installation: absl-py 0.10.0
    Uninstalling absl-py-0.10.0:
      Successfully uninstalled absl-py-0.10.0
     |████████████████████████████████| 7.5MB 11.5MB/s 
  Found existing installation: Django 3.1.3
    Uninstalling Django-3.1.3:
      Successfully uninstalled Django-3.1.3
     |████████████████████████████████| 829kB 11.7MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=b3d5fc0b2f78fbdf1895c65b1184c1050b6861ed9a9e7d61e06a60626ecdf3ad
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existi

     |████████████████████████████████| 757kB 10.5MB/s 
     |████████████████████████████████| 92kB 5.5MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.17.2, but you'll have google-auth 1.18.0 which is incompatible.
  Found existing installation: google-auth 1.17.2
    Uninstalling google-auth-1.17.2:
      Successfully uninstalled google-auth-1.17.2


  Found existing installation: google-auth-oauthlib 0.4.2
    Uninstalling google-auth-oauthlib-0.4.2:
      Successfully uninstalled google-auth-oauthlib-0.4.2
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 3.0MB 11.8MB/s 
  Found existing installation: grpcio 1.33.2
    Uninstalling grpcio-1.33.2:
      Successfully uninstalled grpcio-1.33.2
     |████████████████████████████████| 3.3MB 11.6MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1
     |████████████████████████████████| 92kB 5.6MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: kiwisolver 1.3.1
    Uninstallin

     |████████████████████████████████| 92kB 6.2MB/s 
  Found existing installation: Markdown 3.3.3
    Uninstalling Markdown-3.3.3:
      Successfully uninstalled Markdown-3.3.3
     |████████████████████████████████| 68.7MB 58kB/s 
     |████████████████████████████████| 1.6MB 11.2MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: networkx 2.5
    Uninstalling networkx-2.5:
      Successfully uninstalled networkx-2.5
     |████████████████████████████████| 14.6MB 341kB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.17.2, but you'll have google-auth 1.18.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but 

     |████████████████████████████████| 43.7MB 105kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 71kB 4.9MB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.0 which is incompatible.
  Found existing installation: opt-einsum 3.3.0
    Uninstalling opt-einsum-3.3.0:
      Successfully uninstalled opt-einsum-3.3.0
     |████████████████████████████████| 2.2MB 10.9MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


  Found existing installation: portalocker 2.0.0
    Uninstalling portalocker-2.0.0:
      Successfully uninstalled portalocker-2.0.0
     |████████████████████████████████| 1.3MB 11.4MB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.0 which is incompatible.
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [1]:
#!/bin/bash
!apt-get update
!wget http://download.tensorflow.org/models/deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz
!wget http://download.tensorflow.org/models/deeplabv3_pascal_train_aug_2018_01_04.tar.gz

!mkdir -p models/mobile_net_model/model
!tar xvzf deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz -C models/mobile_net_model/model --strip=1

!mkdir -p models/xception_model/model
!tar xvzf deeplabv3_pascal_train_aug_2018_01_04.tar.gz -C models/xception_model/model --strip=1


!mkdir -p models/basnet
!mkdir -p models/u2net
!mkdir -p models/u2netp

import gdown

url = 'https://drive.google.com/uc?id=1s52ek_4YTDRt_EOkx1FS53u-vJa0c4nu'
output = 'basnet.pth'
gdown.download(url, output, quiet=False)
gdown.cached_download(url, output, postprocess=gdown.extractall)

url = 'https://drive.google.com/uc?id=1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy'
output = 'u2netp.pth'
gdown.download(url, output, quiet=False)
gdown.cached_download(url, output, postprocess=gdown.extractall)

url = 'https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ'
output = 'u2net.pth'
gdown.download(url, output, quiet=False)
gdown.cached_download(url, output, postprocess=gdown.extractall)

!mv basnet.pth ../models/basnet/
!mv u2net.pth ../models/u2net/
!mv u2netp.pth ../models/u2netp/

!cd 
!rm deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz
!rm deeplabv3_pascal_train_aug_2018_01_04.tar.gz
!rm -rf setup*


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu18

Downloading...
From: https://drive.google.com/uc?id=1s52ek_4YTDRt_EOkx1FS53u-vJa0c4nu
To: /content/basnet.pth
348MB [00:10, 32.0MB/s]


File exists: basnet.pth


Downloading...
From: https://drive.google.com/uc?id=1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy
To: /content/u2netp.pth
4.68MB [00:00, 31.1MB/s]


File exists: u2netp.pth


Downloading...
From: https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ
To: /content/u2net.pth
176MB [00:01, 132MB/s]

File exists: u2net.pth
mv: cannot move 'basnet.pth' to '../models/basnet/': No such file or directory


mv: cannot move 'u2net.pth' to '../models/u2net/': No such file or directory
mv: cannot move 'u2netp.pth' to '../models/u2netp/': No such file or directory


# Remover



In [ ]:
import argparse
import os
import gc
import tqdm
import logging
from libs.strings import *
from libs.networks import model_detect
import libs.preprocessing as preprocessing
import libs.postprocessing as postprocessing

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)


def __work_mode__(path: str):
    """Determines the desired mode of operation"""
    if os.path.isfile(path):  # Input is file
        return "file"
    if os.path.isdir(path):  # Input is dir
        return "dir"
    else:
        return "no"


def __save_image_file__(img, file_name, output_path, wmode):
    """
    Saves the PIL image to a file
    :param img: PIL image
    :param file_name: File name
    :param output_path: Output path
    :param wmode: Work mode
    """
    # create output directory if it doesn't exist
    folder = os.path.dirname(output_path)
    if folder != '':
        os.makedirs(folder, exist_ok=True)
    if wmode == "file":
        file_name_out = os.path.basename(output_path)
        if file_name_out == '':
            # Change file extension to png
            file_name = os.path.splitext(file_name)[0] + '.png'
            # Save image
            img.save(os.path.join(output_path, file_name))
            gc.collect()
        else:
            try:
                # Save image
                img.save(output_path)
                gc.collect()
            except OSError as e:
                if str(e) == "cannot write mode RGBA as JPEG":
                    raise OSError("Error! "
                                  "Please indicate the correct extension of the final file, for example: .png")
                else:
                    raise e
    else:
        # Change file extension to png
        file_name = os.path.splitext(file_name)[0] + '.png'
        # Save image
        img.save(os.path.join(output_path, file_name))
        gc.collect()


def process(input_path, output_path, model_name="u2net",
            preprocessing_method_name="bbd-fastrcnn", postprocessing_method_name="rtb-bnb"):
    """
    Processes the file.
    :param input_path: The path to the image / folder with the images to be processed.
    :param output_path: The path to the save location.
    :param model_name: Model to use.
    :param postprocessing_method_name: Method for image preprocessing
    :param preprocessing_method_name: Method for image post-processing
    """
    if input_path is None or output_path is None:
        raise Exception(
            "Bad parameters! Please specify input path and output path.")

    model = model_detect(model_name)  # Load model
    if not model:
        logger.warning("Warning! You specified an invalid model type. "
                       "For image processing, the model with the best processing quality will be used. "
                       "(u2net)")
        # If the model line is wrong, select the model with better quality.
        model_name = "u2net"
        model = model_detect(model_name)  # Load model
    preprocessing_method = preprocessing.method_detect(
        preprocessing_method_name)
    postprocessing_method = postprocessing.method_detect(
        postprocessing_method_name)
    wmode = __work_mode__(input_path)  # Get work mode
    if wmode == "file":  # File work mode
        image = model.process_image(
            input_path, preprocessing_method, postprocessing_method)
        __save_image_file__(image, os.path.basename(
            input_path), output_path, wmode)
    elif wmode == "dir":  # Dir work mode
        # Start process
        files = os.listdir(input_path)
        for file in tqdm.tqdm(files, ascii=True, desc='Remove Background', unit='image'):
            file_path = os.path.join(input_path, file)
            image = model.process_image(
                file_path, preprocessing_method, postprocessing_method)
            __save_image_file__(image, file, output_path, wmode)
    else:
        raise Exception(
            "Bad input parameter! Please indicate the correct path to the file or folder.")
